# Object Following - Live Demo

In this notebook we'll show how you can follow an object with JetBot!  We'll use a pre-trained neural network
that was trained on the [COCO dataset](http://cocodataset.org) to detect 90 different common objects.  These include

* Person (index 0)
* Cup (index 47)

and many others (you can check [this file](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt) for a full list of class indices).  The model is sourced from the [TensorFlow object detection API](https://github.com/tensorflow/models/tree/master/research/object_detection),
which provides utilities for training object detectors for custom tasks also!  Once the model is trained, we optimize it using NVIDIA TensorRT on the Jetson Nano.

This makes the network very fast, capable of real-time execution on Jetson Nano!  We won't run through all of the training and optimization steps in this notebook though.

Anyways, let's get started.  First, we'll want to import the ``ObjectDetector`` class which takes our pre-trained SSD engine.

### Compute detections on single camera image

In [1]:
import sys
import time
import ctypes
import argparse

import numpy as np
import cv2
import pycuda.autoinit  # This is needed for initializing CUDA driver
import pycuda.driver as cuda
import tensorrt as trt

In [2]:
WINDOW_NAME = 'TrtSsdDemo'
INPUT_WH = (300, 300)
OUTPUT_LAYOUT = 7
def preprocess(img):
    """Preprocess an image before SSD inferencing."""
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, INPUT_WH)
    img = img.transpose((2, 0, 1)).astype(np.float32)
    img = (2.0/255.0) * img - 1.0
    return img


def postprocess(img, output, conf_th):
    """Postprocess TRT SSD output."""
    img_h, img_w, _ = img.shape
    boxes, confs, clss = [], [], []
    for prefix in range(0, len(output), OUTPUT_LAYOUT):
        #index = int(output[prefix+0])
        conf = float(output[prefix+2])
        if conf < conf_th:
            continue
        x1 = int(output[prefix+3] * img_w)
        y1 = int(output[prefix+4] * img_h)
        x2 = int(output[prefix+5] * img_w)
        y2 = int(output[prefix+6] * img_h)
        cls = int(output[prefix+1])
        boxes.append((x1, y1, x2, y2))
        confs.append(conf)
        clss.append(cls)
    return boxes, confs, clss

In [3]:
class TrtSSD(object):
    """TrtSSD class encapsulates things needed to run TRT SSD."""

    def _load_plugins(self):
        ctypes.CDLL("ssd/libflattenconcat.so")
        trt.init_libnvinfer_plugins(self.trt_logger, '')

    def _load_engine(self):
        TRTbin = 'ssd/TRT_%s.bin' % self.model
        with open(TRTbin, 'rb') as f, trt.Runtime(self.trt_logger) as runtime:
            return runtime.deserialize_cuda_engine(f.read())

    def _create_context(self):
        for binding in self.engine:
            size = trt.volume(self.engine.get_binding_shape(binding)) * \
                   self.engine.max_batch_size
            host_mem = cuda.pagelocked_empty(size, np.float32)
            cuda_mem = cuda.mem_alloc(host_mem.nbytes)
            self.bindings.append(int(cuda_mem))
            if self.engine.binding_is_input(binding):
                self.host_inputs.append(host_mem)
                self.cuda_inputs.append(cuda_mem)
            else:
                self.host_outputs.append(host_mem)
                self.cuda_outputs.append(cuda_mem)
        return self.engine.create_execution_context()

    def __init__(self, model):
        """Initialize TensorRT plugins, engine and conetxt."""
        self.model = model
        self.trt_logger = trt.Logger(trt.Logger.INFO)
        self._load_plugins()
        self.engine = self._load_engine()

        self.host_inputs = []
        self.cuda_inputs = []
        self.host_outputs = []
        self.cuda_outputs = []
        self.bindings = []
        self.stream = cuda.Stream()
        self.context = self._create_context()

    def __del__(self):
        """Free CUDA memories."""
        del self.stream
        del self.cuda_outputs
        del self.cuda_inputs

    def detect(self, img, conf_th=0.3):
        """Detect objects in the input image."""
        img_resized = preprocess(img)
        np.copyto(self.host_inputs[0], img_resized.ravel())

        cuda.memcpy_htod_async(
            self.cuda_inputs[0], self.host_inputs[0], self.stream)
        self.context.execute_async(
            batch_size=1,
            bindings=self.bindings,
            stream_handle=self.stream.handle)
        cuda.memcpy_dtoh_async(
            self.host_outputs[1], self.cuda_outputs[1], self.stream)
        cuda.memcpy_dtoh_async(
            self.host_outputs[0], self.cuda_outputs[0], self.stream)
        self.stream.synchronize()

        output = self.host_outputs[0]
        return postprocess(img, output, conf_th)


Internally, the ``ObjectDetector`` class uses the TensorRT Python API to execute the engine that we provide.  It also takes care of preprocessing the input to the neural network, as
well as parsing the detected objects.  Right now it will only work for engines created using the ``jetbot.ssd_tensorrt`` package. That package has the utilities for converting
the model from the TensorFlow object detection API to an optimized TensorRT engine.

Next, let's initialize our camera.  Our detector takes 300x300 pixel input, so we'll set this when creating the camera.

> Internally, the Camera class uses GStreamer to take advantage of Jetson Nano's Image Signal Processor (ISP).  This is super fast and offloads
> a lot of the resizing computation from the CPU. 

In [4]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Now, let's execute our network using some camera input.  By default the ``ObjectDetector`` class expects ``bgr8`` format that the camera produces.  However,
you could override the default pre-processing function if your input is in a different format.

In [18]:
import traitlets
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
import cv2

image=camera.value

image_widget = widgets.Image(format='jpeg', width=300, height=300)
width = int(image_widget.width)
height = int(image_widget.height)
display(image_widget)
# detections = model(image)

conf_th = 0.3
model = 'ssd_mobilenet_v2_egohands'

trt_ssd = TrtSSD(model)
# boxes, confs, clss = trt_ssd.detect(image, conf_th)
detections =  trt_ssd.detect(image, conf_th)
print(detections)


# for det in detections[0]:
#     bbox = det['bbox']
#     cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    

image_widget.value = bgr8_to_jpeg(image)

# print(detections)

Image(value=b'', format='jpeg', height='300', width='300')

AttributeError: 'Tensor' object has no attribute 'ravel'

If there are any COCO objects in the camera's field of view, they should now be stored in the ``detections`` variable.

### Display detections in text area

We'll use the code below to print out the detected objects.

In [10]:
from IPython.display import display
import ipywidgets.widgets as widgets


detections_widget = widgets.Textarea()

detections_widget.value = str(detections)

display(detections_widget)

Textarea(value='([(109, 57, 126, 81)], [0.9851575493812561], [1])')

You should see the label, confidence, and bounding box of each object detected in each image.  There's only one image (our camera) in this example. 


To print just the first object detected in the first image, we could call the following

> This may throw an error if no objects are detected

In [11]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

(109, 57, 126, 81)


### Control robot to follow central object

Now we want our robot to follow an object of the specified class.  To do this we'll do the following

1.  Detect objects matching the specified class
2.  Select object closest to center of camera's field of vision, this is the 'target' object
3.  Steer robot towards target object, otherwise wander
4.  If we're blocked by an obstacle, turn left

We'll also create some widgets that we'll use to control the target object label, the robot speed, and
a "turn gain", that will control how fast the robot turns based off the distance between the target object
and the center of the robot's field of view. 


First, let's load our collision detection model.  The pre-trained model is stored in this directory as a convenience, but if you followed
the collision avoidance example you may want to use that model if it's better tuned for your robot's environment.

In [12]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 7)
collision_model.load_state_dict(torch.load('best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great, now let's initialize our robot so we can control the motors.

In [13]:
from jetbot import Robot

robot = Robot()

Finally, let's display all the control widgets and connect the network execution function to the camera updates.

In [14]:
from jetbot import bgr8_to_jpeg
import time

turn_left = 1
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
red_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='red')
blue_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blue')
green_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='green')
yellow_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='yellow')
free_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='free')

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    widgets.HBox([red_widget,blue_widget]),
    widgets.HBox([green_widget,yellow_widget]),
    free_widget,
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    global turn_left
    image = change['new']
    print("in execute")
    
    # execute collision model to determine if blocked
    collision_output = collision_model(preprocess(image)).detach().cpu()
    y = F.softmax(collision_output.flatten(), dim=0)
    y = y.flatten()
#     prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    prob_blocked = float(y[0])
    prob_blue = float(y[1])
    prob_free = float(y[2])
    prob_green = float(y[3])
    prob_hit = float(y[4])
    prob_red = float(y[5])
    prob_yellow = float(y[6])
#     blocked_widget.value = prob_blocked
#     red_widget.value = prob_red
#     blue_widget.value = prob_blue
#     green_widget.value = prob_green
#     yellow_widget.value = prob_yellow
#     free_widget.value = prob_free
    print(prob_blocked)
    image_widget.value = bgr8_to_jpeg(image)
    
    # turn left if blocked
    if prob_free > 0.4:
        robot.forward(0.2)
    elif prob_red > 0.5 or prob_yellow > 0.5 or prob_green > 0.5 or prob_blue > 0.5 :
        robot.left(0.1)
        image_widget.value = bgr8_to_jpeg(image)
        return
    elif prob_blocked > 0.6:
#         robot.backward(0.1)
#         time.sleep(0.25)
        robot.left(0.2)
        return
    elif prob_hit > 0.5:
        print("Entered into hit")
        camera.unobserve_all()
        time.sleep(1.0)
        robot.stop()
#         robot.backward(0.2)
        time.sleep(0.5)
        robot.forward(0.4)
        time.sleep(0.5)
        robot.stop()
        time.sleep(2)
        camera.observe(execute, names='value')
        print("Exiting hit")
        return
        
    # compute all detected objects
#     detections = model(image)
    bbox, confs, clss = trt_ssd.detect(image, conf_th)
    
    # draw all detections on image
#     for det in detections[0]:
#         bbox = det['bbox']
#         cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # select detections that match selected class label
#     matching_detections = [d for d in detections[0] if d['label'] == 37]
    
    # get detection closest to center of field of view and draw it
#     det = closest_detection(matching_detections)
#     if det is not None:
#         bbox = det['bbox']
    if bbox is not None:
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    
        
    # otherwise go forward if no target detected
    if bbox is None:
#         robot.left(0.2)
        robot.forward(0.2)
        
    # otherwsie steer towards target
    else:
        print("Entering ball detect")
        # move robot forward and steer proportional target's x-distance from center
        camera.unobserve_all()
        time.sleep(0.5)
        center = detection_center(bbox)
        print(center)
#         turn_g = turn_gain_widget.value
        turn_g = 0.30
        speed_g = speed_widget.value
        robot.set_motors(
            float(speed_g + turn_g * center[0]),
            float(speed_g - turn_g * center[0])
        )
        robot.stop()
        time.sleep(2)
        camera.observe(execute, names='value')
        print("Exiting ball detect")
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

in execute
0.878229558467865


Call the block below to connect the execute function to each camera frame update.

In [15]:
camera.unobserve_all()
camera.observe(execute, names='value')

Awesome!  If the robot is not blocked you should see boxes drawn around the detected objects in blue.  The target object (which the robot follows) will be displayed in green.

The robot should steer towards the target when it is detected.  If it is blocked by an object it will simply turn left.

You can call the code block below to manually disconnect the processing from the camera and stop the robot.

In [16]:


camera.unobserve_all()
time.sleep(1.0)
robot.stop()